In [1]:
# Library Imports

import gc
import math
import time
import traceback
import warnings
import urllib.parse
import pandas as pd
import numpy as np
import connectorx as cx
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.types import Integer,Float
import psycopg2.extras as extras
warnings.filterwarnings("ignore")

In [2]:
# DATABASE PARAMETER

database = 'isgpostgres'
user = 'postgres'
password = '123@NjMpTs'
host = '20.62.75.39'
port = '9030'
connetion = psycopg2.connect(database = database, user = user ,password = password, host = host,port = port)

In [3]:
# GLOBAL VARIABLE PARAMETERs
schema_name = 'tm_new_data'
volume_table  = 'tm_new_data.nys_hour_8_volume_240626'
connected_table = 'tm_new_data.ny_connected_osm_master_231211'
geh_table = 'nys_hour_8_volume_240626_adjacent_geh_check'

In [4]:
# GET BASE DATA FOR AVAILABLE VOLUME

sql_command = f"""select distinct isg_osm_id,prev_connected_count,source,highway_id,volume,
                    single_next_ids,multi_next_ids,single_prev_ids,multi_prev_ids
                    from (select A.*,B.* from  
                    (select * from {volume_table} where volume > 0 and prev_connected_count=1 and prev_ncc = 1) A ,
                    {connected_table} B 
                    where A.isg_osm_id = B.current_id)foo"""    

# GET PROCESS LIST

All_Data_df = pd.read_sql(sql_command, connetion)
Updated_Available_LST = np.unique(list(set(All_Data_df.to_dict(orient='list')['isg_osm_id'])))

In [5]:
remove_from_prev_geh_df = pd.read_sql(f'''select distinct conn_isg_osm_array[1]
                                        as removal_ids_prev_geh from {schema_name}.{geh_table} where conn_tag = 'single'
                                        union
                                        select distinct isg_osm_id as removal_ids_prev_geh from {schema_name}.{geh_table}
                                        where source = 'multi_prev'
                                        ''',connetion)

remove_from_prev_geh_list = remove_from_prev_geh_df['removal_ids_prev_geh'].unique()
prev_geh_process_list = list(set(Updated_Available_LST) - set(remove_from_prev_geh_list))

In [6]:
len_Updated_Available_LST = len(Updated_Available_LST)
print("Available Volume :",len_Updated_Available_LST)

print("remove_from_prev_geh_list :",len(remove_from_prev_geh_list)) 

len_prev_geh_process_list = len(prev_geh_process_list)
print("prev_geh_preocess_list :",len_prev_geh_process_list)

Available Volume : 22855
remove_from_prev_geh_list : 619
prev_geh_preocess_list : 22236


In [7]:
def geh_check(current_vol,next_vol):
    return round(math.sqrt(2*((current_vol-next_vol)*(current_vol-next_vol))/(current_vol+next_vol)),2)

In [8]:
Missing_IDS = []
error_dict = {}
iteration='Original'

augmented_df = pd.DataFrame({'isg_osm_id':[],'conn_link_array':[],'connected_count':[],
                             'source':[], 'conn_src_array':[],
                             'volume':[],'total_conn_vol':[],'conn_vol_array':[],
                             'conn_tag':[],'iteration':[],'geh':[]},dtype='int64')

# augmented_df = pd.DataFrame(dtype = 'int64')
start = time.time()

# PREV GEH CALCULATION
counter = 0  
for search_osm_id in prev_geh_process_list:
    try:
        
        cur_df = All_Data_df.loc[All_Data_df['isg_osm_id'] == search_osm_id].reset_index(drop = True)
        single_prev_ids_lst = cur_df['single_prev_ids'].iloc[0]
        cur_volume = cur_df['volume'].iloc[0]
        connected_count = cur_df['prev_connected_count'].iloc[0]
        source = cur_df['source'].iloc[0]
        common_elements = list(set(single_prev_ids_lst) & set(Updated_Available_LST))
            
        if len(common_elements) == 0:
            #print("prev")
        
            multi_prev_ids_lst = cur_df['multi_prev_ids'].iloc[0]
            multi_prev_data=All_Data_df.loc[All_Data_df['isg_osm_id'].isin(multi_prev_ids_lst)].reset_index(drop = True)
            conn_src_array =  multi_prev_data[['isg_osm_id','source']].drop_duplicates()['source'].to_list()

            if len(multi_prev_data) == len(multi_prev_ids_lst) and len(multi_prev_ids_lst) > 1:
                
                prev_volume = multi_prev_data['volume'].sum().item()
                geh=geh_check(cur_volume,prev_volume)
                multi_prev_df = pd.DataFrame()               
                multi_prev_df['isg_osm_id'] = [int(search_osm_id)]
                multi_prev_df['conn_link_array'] = [multi_prev_ids_lst]
                multi_prev_df['connected_count'] = [int(connected_count)]
                multi_prev_df['source'] = [source]
                multi_prev_df['conn_src_array'] = [conn_src_array]
                multi_prev_df['volume'] = [int(cur_volume)]
                multi_prev_df['total_conn_vol'] = [int(prev_volume)]
                multi_prev_df['conn_vol_array'] = [multi_prev_data['volume'].unique().tolist()]
                multi_prev_df['conn_tag'] = ['multi_prev']
                multi_prev_df['iteration'] =iteration
                multi_prev_df['geh'] = [float(geh)]
                
                augmented_df=pd.concat([augmented_df, multi_prev_df], ignore_index=True)

            if len(multi_prev_data) < len(multi_prev_ids_lst) and len(multi_prev_ids_lst) > 1 and len(multi_prev_data) >0:
                
                prev_volume = multi_prev_data['volume'].sum().item()
                if prev_volume > cur_volume:
                    geh=geh_check(cur_volume,prev_volume)
                    multi_prev_df = pd.DataFrame()
                    multi_prev_df['isg_osm_id'] = [int(search_osm_id)]
                    multi_prev_df['conn_link_array'] = [multi_prev_data['isg_osm_id'].unique().tolist()]
                    multi_prev_df['connected_count'] = [int(connected_count)]
                    multi_prev_df['source'] = [source]
                    multi_prev_df['conn_src_array'] = [conn_src_array]
                    multi_prev_df['volume'] = [int(cur_volume)]
                    multi_prev_df['total_conn_vol'] = [int(prev_volume)]
                    multi_prev_df['conn_vol_array'] = [multi_prev_data['volume'].unique().tolist()]
                    multi_prev_df['conn_tag'] = ['multi_prev']
                    multi_prev_df['iteration'] =iteration
                    multi_prev_df['geh'] = [float(geh)]

                    augmented_df=pd.concat([augmented_df, multi_prev_df], ignore_index=True)

        counter = counter + 1
        print("Processed ID :",counter)
        
    except:
        error_dict[str(search_osm_id)] = str(traceback.format_exc())
        Missing_IDS.append(search_osm_id)
        print("Some ID Missed")
        continue

end = time.time()

Processed ID : 1
Processed ID : 2
Processed ID : 3
Processed ID : 4
Processed ID : 5
Processed ID : 6
Processed ID : 7
Processed ID : 8
Processed ID : 9
Processed ID : 10
Processed ID : 11
Processed ID : 12
Processed ID : 13
Processed ID : 14
Processed ID : 15
Processed ID : 16
Processed ID : 17
Processed ID : 18
Processed ID : 19
Processed ID : 20
Processed ID : 21
Processed ID : 22
Processed ID : 23
Processed ID : 24
Processed ID : 25
Processed ID : 26
Processed ID : 27
Processed ID : 28
Processed ID : 29
Processed ID : 30
Processed ID : 31
Processed ID : 32
Processed ID : 33
Processed ID : 34
Processed ID : 35
Processed ID : 36
Processed ID : 37
Processed ID : 38
Processed ID : 39
Processed ID : 40
Processed ID : 41
Processed ID : 42
Processed ID : 43
Processed ID : 44
Processed ID : 45
Processed ID : 46
Processed ID : 47
Processed ID : 48
Processed ID : 49
Processed ID : 50
Processed ID : 51
Processed ID : 52
Processed ID : 53
Processed ID : 54
Processed ID : 55
Processed ID : 56
P

Processed ID : 2116
Processed ID : 2117
Processed ID : 2118
Processed ID : 2119
Processed ID : 2120
Processed ID : 2121
Processed ID : 2122
Processed ID : 2123
Processed ID : 2124
Processed ID : 2125
Processed ID : 2126
Processed ID : 2127
Processed ID : 2128
Processed ID : 2129
Processed ID : 2130
Processed ID : 2131
Processed ID : 2132
Processed ID : 2133
Processed ID : 2134
Processed ID : 2135
Processed ID : 2136
Processed ID : 2137
Processed ID : 2138
Processed ID : 2139
Processed ID : 2140
Processed ID : 2141
Processed ID : 2142
Processed ID : 2143
Processed ID : 2144
Processed ID : 2145
Processed ID : 2146
Processed ID : 2147
Processed ID : 2148
Processed ID : 2149
Processed ID : 2150
Processed ID : 2151
Processed ID : 2152
Processed ID : 2153
Processed ID : 2154
Processed ID : 2155
Processed ID : 2156
Processed ID : 2157
Processed ID : 2158
Processed ID : 2159
Processed ID : 2160
Processed ID : 2161
Processed ID : 2162
Processed ID : 2163
Processed ID : 2164
Processed ID : 2165


Processed ID : 2530
Processed ID : 2531
Processed ID : 2532
Processed ID : 2533
Processed ID : 2534
Processed ID : 2535
Processed ID : 2536
Processed ID : 2537
Processed ID : 2538
Processed ID : 2539
Processed ID : 2540
Processed ID : 2541
Processed ID : 2542
Processed ID : 2543
Processed ID : 2544
Processed ID : 2545
Processed ID : 2546
Processed ID : 2547
Processed ID : 2548
Processed ID : 2549
Processed ID : 2550
Processed ID : 2551
Processed ID : 2552
Processed ID : 2553
Processed ID : 2554
Processed ID : 2555
Processed ID : 2556
Processed ID : 2557
Processed ID : 2558
Processed ID : 2559
Processed ID : 2560
Processed ID : 2561
Processed ID : 2562
Processed ID : 2563
Processed ID : 2564
Processed ID : 2565
Processed ID : 2566
Processed ID : 2567
Processed ID : 2568
Processed ID : 2569
Processed ID : 2570
Processed ID : 2571
Processed ID : 2572
Processed ID : 2573
Processed ID : 2574
Processed ID : 2575
Processed ID : 2576
Processed ID : 2577
Processed ID : 2578
Processed ID : 2579


Processed ID : 2944
Processed ID : 2945
Processed ID : 2946
Processed ID : 2947
Processed ID : 2948
Processed ID : 2949
Processed ID : 2950
Processed ID : 2951
Processed ID : 2952
Processed ID : 2953
Processed ID : 2954
Processed ID : 2955
Processed ID : 2956
Processed ID : 2957
Processed ID : 2958
Processed ID : 2959
Processed ID : 2960
Processed ID : 2961
Processed ID : 2962
Processed ID : 2963
Processed ID : 2964
Processed ID : 2965
Processed ID : 2966
Processed ID : 2967
Processed ID : 2968
Processed ID : 2969
Processed ID : 2970
Processed ID : 2971
Processed ID : 2972
Processed ID : 2973
Processed ID : 2974
Processed ID : 2975
Processed ID : 2976
Processed ID : 2977
Processed ID : 2978
Processed ID : 2979
Processed ID : 2980
Processed ID : 2981
Processed ID : 2982
Processed ID : 2983
Processed ID : 2984
Processed ID : 2985
Processed ID : 2986
Processed ID : 2987
Processed ID : 2988
Processed ID : 2989
Processed ID : 2990
Processed ID : 2991
Processed ID : 2992
Processed ID : 2993


Processed ID : 3356
Processed ID : 3357
Processed ID : 3358
Processed ID : 3359
Processed ID : 3360
Processed ID : 3361
Processed ID : 3362
Processed ID : 3363
Processed ID : 3364
Processed ID : 3365
Processed ID : 3366
Processed ID : 3367
Processed ID : 3368
Processed ID : 3369
Processed ID : 3370
Processed ID : 3371
Processed ID : 3372
Processed ID : 3373
Processed ID : 3374
Processed ID : 3375
Processed ID : 3376
Processed ID : 3377
Processed ID : 3378
Processed ID : 3379
Processed ID : 3380
Processed ID : 3381
Processed ID : 3382
Processed ID : 3383
Processed ID : 3384
Processed ID : 3385
Processed ID : 3386
Processed ID : 3387
Processed ID : 3388
Processed ID : 3389
Processed ID : 3390
Processed ID : 3391
Processed ID : 3392
Processed ID : 3393
Processed ID : 3394
Processed ID : 3395
Processed ID : 3396
Processed ID : 3397
Processed ID : 3398
Processed ID : 3399
Processed ID : 3400
Processed ID : 3401
Processed ID : 3402
Processed ID : 3403
Processed ID : 3404
Processed ID : 3405


...

Processed ID : 33144
Processed ID : 33145
Processed ID : 33146
Processed ID : 33147
Processed ID : 33148
Processed ID : 33149
Processed ID : 33150
Processed ID : 33151
Processed ID : 33152
Processed ID : 33153
Processed ID : 33154
Processed ID : 33155
Processed ID : 33156
Processed ID : 33157
Processed ID : 33158
Processed ID : 33159
Processed ID : 33160
Processed ID : 33161
Processed ID : 33162
Processed ID : 33163
Processed ID : 33164
Processed ID : 33165
Processed ID : 33166
Processed ID : 33167
Processed ID : 33168
Processed ID : 33169
Processed ID : 33170
Processed ID : 33171
Processed ID : 33172
Processed ID : 33173
Processed ID : 33174
Processed ID : 33175
Processed ID : 33176
Processed ID : 33177
Processed ID : 33178
Processed ID : 33179
Processed ID : 33180
Processed ID : 33181
Processed ID : 33182
Processed ID : 33183
Processed ID : 33184
Processed ID : 33185
Processed ID : 33186
Processed ID : 33187
Processed ID : 33188
Processed ID : 33189
Processed ID : 33190
Processed ID 

Processed ID : 33541
Processed ID : 33542
Processed ID : 33543
Processed ID : 33544
Processed ID : 33545
Processed ID : 33546
Processed ID : 33547
Processed ID : 33548
Processed ID : 33549
Processed ID : 33550
Processed ID : 33551
Processed ID : 33552
Processed ID : 33553
Processed ID : 33554
Processed ID : 33555
Processed ID : 33556
Processed ID : 33557
Processed ID : 33558
Processed ID : 33559
Processed ID : 33560
Processed ID : 33561
Processed ID : 33562
Processed ID : 33563
Processed ID : 33564
Processed ID : 33565
Processed ID : 33566
Processed ID : 33567
Processed ID : 33568
Processed ID : 33569
Processed ID : 33570
Processed ID : 33571
Processed ID : 33572
Processed ID : 33573
Processed ID : 33574
Processed ID : 33575
Processed ID : 33576
Processed ID : 33577
Processed ID : 33578
Processed ID : 33579
Processed ID : 33580
Processed ID : 33581
Processed ID : 33582
Processed ID : 33583
Processed ID : 33584
Processed ID : 33585
Processed ID : 33586
Processed ID : 33587
Processed ID 

Processed ID : 33933
Processed ID : 33934
Processed ID : 33935
Processed ID : 33936
Processed ID : 33937
Processed ID : 33938
Processed ID : 33939
Processed ID : 33940
Processed ID : 33941
Processed ID : 33942
Processed ID : 33943
Processed ID : 33944
Processed ID : 33945
Processed ID : 33946
Processed ID : 33947
Processed ID : 33948
Processed ID : 33949
Processed ID : 33950
Processed ID : 33951
Processed ID : 33952
Processed ID : 33953
Processed ID : 33954
Processed ID : 33955
Processed ID : 33956
Processed ID : 33957
Processed ID : 33958
Processed ID : 33959
Processed ID : 33960
Processed ID : 33961
Processed ID : 33962
Processed ID : 33963
Processed ID : 33964
Processed ID : 33965
Processed ID : 33966
Processed ID : 33967
Processed ID : 33968
Processed ID : 33969
Processed ID : 33970
Processed ID : 33971
Processed ID : 33972
Processed ID : 33973
Processed ID : 33974
Processed ID : 33975
Processed ID : 33976
Processed ID : 33977
Processed ID : 33978
Processed ID : 33979
Processed ID 

Processed ID : 34327
Processed ID : 34328
Processed ID : 34329
Processed ID : 34330
Processed ID : 34331
Processed ID : 34332
Processed ID : 34333
Processed ID : 34334
Processed ID : 34335
Processed ID : 34336
Processed ID : 34337
Processed ID : 34338
Processed ID : 34339
Processed ID : 34340
Processed ID : 34341
Processed ID : 34342
Processed ID : 34343
Processed ID : 34344
Processed ID : 34345
Processed ID : 34346
Processed ID : 34347
Processed ID : 34348
Processed ID : 34349
Processed ID : 34350
Processed ID : 34351
Processed ID : 34352
Processed ID : 34353
Processed ID : 34354
Processed ID : 34355
Processed ID : 34356
Processed ID : 34357
Processed ID : 34358
Processed ID : 34359
Processed ID : 34360
Processed ID : 34361
Processed ID : 34362
Processed ID : 34363
Processed ID : 34364
Processed ID : 34365
Processed ID : 34366
Processed ID : 34367
Processed ID : 34368
Processed ID : 34369
Processed ID : 34370
Processed ID : 34371
Processed ID : 34372
Processed ID : 34373
Processed ID 

Processed ID : 34723
Processed ID : 34724
Processed ID : 34725
Processed ID : 34726
Processed ID : 34727
Processed ID : 34728
Processed ID : 34729
Processed ID : 34730
Processed ID : 34731
Processed ID : 34732
Processed ID : 34733
Processed ID : 34734
Processed ID : 34735
Processed ID : 34736
Processed ID : 34737
Processed ID : 34738
Processed ID : 34739
Processed ID : 34740
Processed ID : 34741
Processed ID : 34742
Processed ID : 34743
Processed ID : 34744
Processed ID : 34745
Processed ID : 34746
Processed ID : 34747
Processed ID : 34748
Processed ID : 34749
Processed ID : 34750
Processed ID : 34751
Processed ID : 34752
Processed ID : 34753
Processed ID : 34754
Processed ID : 34755
Processed ID : 34756
Processed ID : 34757
Processed ID : 34758
Processed ID : 34759
Processed ID : 34760
Processed ID : 34761
Processed ID : 34762
Processed ID : 34763
Processed ID : 34764
Processed ID : 34765
Processed ID : 34766
Processed ID : 34767
Processed ID : 34768
Processed ID : 34769
Processed ID 

In [12]:
print("TIME TAKEN To CALCULATE GEH : ",(end - start))

TIME TAKEN To CALCULATE GEH :  90.18413591384888


In [13]:
print(len(augmented_df['isg_osm_id'].unique()))
print(len(augmented_df['isg_osm_id']))
print(len(Missing_IDS))
print(len(error_dict.keys()))

5
5
0
0


In [14]:
augmented_df

isg_osm_id              conn_link_array  connected_count    source  \
0  734275243002               [122633429001]                1  Original   
1  788253907001               [949990384001]                1  Original   
2   46201646001                [33117932001]                1  Original   
3  172158799001   [65678368001, 65678249001]                1  Original   
4   46177182001  [32922459001, 533472710001]                1  Original   

         conn_src_array  volume  total_conn_vol conn_vol_array    conn_tag  \
0            [Original]     420             449          [449]  multi_prev   
1            [Original]    3146            3224         [3224]  multi_prev   
2            [Original]    4068            4248         [4248]  multi_prev   
3  [Original, Original]     673             826     [669, 157]  multi_prev   
4  [Original, Original]    2295            2322    [408, 1914]  multi_prev   

  iteration   geh  
0  Original  1.39  
1  Original  1.38  
2  Original  2.79  
3  Original  5.59  
4  Original  0.56

In [16]:
# Insert Data Into DB
encoded_password = urllib.parse.quote(password)

connection_string = "postgresql://" + user + ":" + encoded_password + "@" + host + ":" + port + "/" + database
engine = create_engine(connection_string)

augmented_df.to_sql('adjacent_geh_script',engine,if_exists='replace',schema=schema_name, chunksize=10000,index=False,method='multi')

5